# Getting Started with TensorFlow

In [2]:
import numpy as np
import tensorflow as tf

TensorFlow Core programs consist of two discrete sections:
* Building the computational graph
* Running the computational graph

A computational graph is a series of TensorFlow operations arranged into a graph of nodes.

### Building a simple computational graph

Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a constant. Like all Tensorflow constants, it takes no inputs, and it outputs a value it stores internally. We create two floating point tensors node1 and node2 as follows:

In [3]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)
print(node1,node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


We do not get the values as we would expect. Instead we get nodes which when evaluated would produce the values respectively. For this we need to run the graph in a **session**. A session encapsulates the state and control of the TensorFlow runtime.

The following code creates a session object and then invokes its run method to run enough of the computational graph to evaluate node1 and node2.

In [4]:
sess = tf.Session()
print(sess.run([node1,node2]))

[3.0, 4.0]


More complicated computations can be performed by combining Tensor nodes with operations. Operations are also nodes. For example we can add two constant nodes and produce a new graph as follows:

In [5]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


TensorFlow provides a utility board called TensorBoard that can display a picture of the computational graph. A graph can also be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to add a value later on.

In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a+b

The preceding three lines are a bit like a function or a lambda in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict parameter to specify Tensors that provide concrete values to these placeholders

In [7]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


We can make the computational graph more complex by adding another operation.

In [8]:
add_and_triple = adder_node*3.
print(sess.run(add_and_triple,{a:3,b:4.5}))

22.5


In machine learning, we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. **Variables** allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [9]:
W = tf.Variable([.3],tf.float32)
b = tf.Variable([-.3],tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

Constants are initialized when we call tf.constant, and their value can never change. By contrast, variables are not initialized when we call tf.variable. To initialize all the variables in a Tensorflow program, you must explicitly call a special operation as follows:

In [10]:
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
init = tf.global_variables_initializer()
sess.run(init)

It is important to realize that init is a handle to the TensorFlow sub-graph that initialzes all the global variables. Until we call sess.run, the variables are uninitialized.

Since x is placeholder, we can evaluate linear_model for several values of x simultaneously as follows:

In [11]:
print(sess.run(linear_model,{x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


We have created a model, but we do not know how good it is yet. To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function.

A loss function measures how far apart the current model is from provided data. We will use the standard loss model for linear regression.

In [12]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

23.66


We can improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed by using operations like tf.assign. For example, W=-1 and b=1 are the optimal parameters for our model. We can change W and b accordingly:

In [13]:
fixW = tf.assign(W,[-1.])
fixb = tf.assign(b,[1.])
sess.run([fixW,fixb])
print(sess.run(loss,{x:[1,2,3,4],y:[0,-1,-2,-3]}))

0.0


Well here we guessed the correct values of both W and b to obtain perfect results. But in machine learning, we need to automate this process of finding the optimal value of the parameters.

### tf.train API

TensorFlow provides **optimizers** that slowly change each variable in order to minimize the loss function. The simplest optimizer is **gradient descent**. TensorFlow can automatically produce derivatives given only a description of the model using the function **tf.gradients**. For simplicity, optimizers does this for us:

In [14]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [15]:
sess.run(init) #reset the value to incorrect defaults
for i in range(1000):
    sess.run(train,{x:x_train,y:y_train})
    
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [16]:
# Evaluating training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


The above code amounts to a very simple machine learning model. TensorFlow provides higher levels of abstractions for common patterns, structures and functionality.

### tf.contrib.learn

tf.contrib.learn is a high-level TensorFlow Library that simplifies the mechanics of machine learning. The basic usage is given below:

In [17]:
# Declare a list of features. We only have one real valued feature in this case. There are many other types of column.
features = [tf.contrib.layers.real_valued_column("x",dimension=1)]

An estimator is the front end to invoke training (fitting) and evaluation (inference). The following code provides an estimator that does linear regression.

In [18]:
estimator = tf.contrib.learn.LinearRegressor(feature_columns = features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B3320FBBE0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}


Tensorflow provides many helper methods to read and set up data sets. Here we use 'numpy_input_fn'. We have to tell the function how many batches of data (num_epochs) we want and how big each batch should be:

In [21]:
x = np.array([1.,2.,3.,4.])
y = np.array([0.,-1.,-2.,-3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,num_epochs=1000)

We can invoke 1000 training steps by invoking the 'fit' method and passing the training data set.

In [22]:
estimator.fit(input_fn=input_fn, steps=1000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\karti\AppData\Local\Temp\tmpj7pz9uq2\model.ckpt.
INFO:tensorflow:loss = 1.5, step = 1
INFO:tensorflow:global_step/sec: 1066.53
INFO:tensorflow:loss = 0.0145579, step = 101 (0.094 sec)
INFO:tensorflow:global_step/sec: 1279.89
INFO:tensorflow:loss = 0.00131965, step = 201 (0.078 sec)
INFO:tensorflow:global_step/sec: 1279.87
INFO:tensorflow:loss = 0.000655074, step = 301 (0.078 sec)
INFO:tensorflow:global_step/sec: 2131.62
INFO:tensorflow:loss = 0.000253548, step = 401 (0.063 sec)
INFO:tensorflow:global_step/sec: 1577.62
INFO:tensorflow:loss = 4.89674e-05, step = 501 (

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000001B32EE06518>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [23]:
# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=input_fn))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2018-04-14-02:12:54
INFO:tensorflow:Restoring parameters from C:\Users\karti\AppData\Local\Temp\tmpj7pz9uq2\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-14-02:12:55
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 5.80248e-09
{'loss': 5.8024816e-09, 'global_step': 1000}
